In [17]:
import urllib.request as ur # import para obtener la página web 
import re                   # import para utilizar expresiones regulares 

# Obtener página web con incidencias de Madrid 
f = ur.urlopen('http://infocar.dgt.es/etraffic/Incidencias?ca=13&provIci=&caracter=acontecimiento&accion_consultar=Consultar&IncidenciasRETENCION=IncidenciasRETENCION&IncidenciasPUERTOS=IncidenciasPUERTOS&IncidenciasMETEOROLOGICA=IncidenciasMETEOROLOGICA&IncidenciasEVENTOS=IncidenciasEVENTOS&IncidenciasOTROS=IncidenciasOTROS&IncidenciasRESTRICCIONES=IncidenciasRESTRICCIONES&ordenacion=fechahora_ini-DESC')
s = f.read().decode()
f.close()

print(s)












<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="es" xml:lang="es" xmlns="http://www.w3.org/1999/xhtml">
  	<head>
	   	<base href="http://infocar.dgt.es:80/etraffic/" />
	   	<link rel="icon" type="image/png" href="images/favicon.ico" />
	    
	    <title>Incidencias de circulación</title>
	    
	   	<script type="text/javascript" src="js/common.js?fecha=05032019"></script>
	   	<script type="text/javascript" src="js/commons.js?fecha=05032019"></script>
	  	<script type="text/javascript" src="js/ajax.js?fecha=05032019"></script>
	   	<script type="text/javascript" src="js/config/textos_es_ES.js?fecha=05032019"></script>
	   	<link rel="stylesheet" type="text/css" href="css/style.css?fecha=05032019" media="all" />
		<link rel="stylesheet" type="text/css" href="css/fonts.css?fecha=05032019" media="all" />
		<link rel="stylesheet" type="text/css" href="css/niftyCorners.css" media="all" />
	   

In [12]:
# Método para eliminar tags html que no nos interesan con ER 
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,'',raw_html)
    return cleantext

In [56]:
from pprint import pprint     # import para hacer prints boninitos 

ss = s.split("<tr")


# Obtener las incidencias que tienen hora fin 
print("Incidencias con hora fin")
confin = re.findall("<tr.*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?",s,re.DOTALL)
for x in ss:
    confin = re.findall(".*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?tr>",x,re.DOTALL)
    pprint(confin)

# Obtener todas las incidencias 
print("\nTodas las incidencias")
conjunto = re.findall("<tr.*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d)?.*?(\d\d/\d\d/\d\d\d\d)?.*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?",s,re.DOTALL)
pprint(conjunto)



Incidencias con hora fin
[]
[]
[]
[]
[('06:00',
  '05/03/2019',
  '23:00',
  '05/03/2019',
  'M-111',
  'RESTRICCIONES A MERCANCÍAS PELIGROSAS</b> por <b>CIRCULACIÓN</b> en<b> '
  'Túnel Barajas</b>: </span> <span style="margin-top:10px; float:left; '
  'clear:both">  -  La carretera  <b> <span style="color:#ab3000">M-111 '
  '</span> </b> a la altura de <b>MADRID (MADRID)</b> desde el <b> km 1.1</b> '
  'al <b> km 5</b> sentido <b>AMBOS SENTIDOS')]

Todas las incidencias
[('15:31',
  '05/03/2019',
  '',
  '',
  'M-40',
  'RETENCIÓN / CONGESTIÓN</b> por <b>CIRCULACIÓN</b> con circulación '
  'irregular  en<b></b>: </span> <span style="margin-top:10px; float:left; '
  'clear:both">  - La AUTOPISTA / AUTOVÍA <b> <span style="color:#ab3000">M-40 '
  '</span> </b> a la altura de <b>MADRID (MADRID)</b> desde el <b> km 31</b> '
  'al <b> km 30</b> sentido <b>CRECIENTE DE LA KILOMETRACIÓN</b> hacia <b>A-5'),
 ('14:23',
  '05/03/2019',
  '',
  '',
  'M-40',
  'RETENCIÓN / CONGESTIÓN</b> por <b

In [19]:
diccionario = {} # variable que se utilizará para almacenar las incidencias una sola vez 

for x in range(len(confin)):
    identificador = confin[x][0] + confin[x][1] + confin[x][4]  # concatenación de valores para crear identificador para diccionario
    diccionario.update({identificador : confin[x]}) # agregación de valores al diccionario

for x in range(len(conjunto)):
    identificador = conjunto[x][0] + conjunto[x][1] + conjunto[x][4] # concatenación de valores para crear identificador para diccionario
    if diccionario.get(identificador) == None:    # En caso de que el identificador exista, salta este paso
        diccionario.update({identificador : conjunto[x]}) # Si el identificador no existe, se crea un nuevo elemento en el diccionario
'''
Para todas las carreteras
'''
        
for x in diccionario:
    valores = diccionario.get(x) # obtener el valor correspondiente para el identificador 
    fin = "" # declarar variable para mensaje en caso de que la incidencia tenga hora fin
    cleantext =  cleanhtml(valores[5]) # quitar las etiquetas de html del texto que describe la incidencia 
    if valores[2] != '':  # En caso de que exista una hora fin
        mensaje = "Hubo un atasco en la carretera " + valores[4]
        fin = " el cual terminó a las " + valores[2] + " el día " + valores[3]
    else: 
        mensaje = "Existe un atasco en la carretera " + valores[4]
    mensaje = mensaje + " que inicio a las " + valores[0] + " el día " + valores[1]
    mensaje = mensaje + fin
    mensaje = mensaje + " con la siguientes características:\n" +  cleantext
    
    print(mensaje+"\n")
    


Existe un atasco en la carretera M-40 que inicio a las 15:31 el día 05/03/2019 con la siguientes características:
RETENCIÓN / CONGESTIÓN por CIRCULACIÓN con circulación irregular  en:    - La AUTOPISTA / AUTOVÍA  M-40   a la altura de MADRID (MADRID) desde el  km 31 al  km 30 sentido CRECIENTE DE LA KILOMETRACIÓN hacia A-5

Existe un atasco en la carretera M-40 que inicio a las 14:23 el día 05/03/2019 con la siguientes características:
RETENCIÓN / CONGESTIÓN por CIRCULACIÓN con circulación irregular  en:    - La AUTOPISTA / AUTOVÍA  M-40   a la altura de MADRID (MADRID) desde el  km 12.9 al  km 10 sentido CRECIENTE DE LA KILOMETRACIÓN hacia A-3

Existe un atasco en la carretera M-111 que inicio a las 06:00 el día 05/03/2019 con la siguientes características:
RESTRICCIONES A MERCANCÍAS PELIGROSAS por CIRCULACIÓN en Túnel Barajas:    -  La carretera   M-111   a la altura de MADRID (MADRID) desde el  km 1.1 al  km 5 sentido AMBOS SENTIDOS



In [16]:
'''
Para la carretera M-40 
'''

for x in diccionario:
    if "M-40" in x: 
        valores = diccionario.get(x) # obtener el valor correspondiente para el identificador 
        fin = "" # declarar variable para mensaje en caso de que la incidencia tenga hora fin
        cleantext =  cleanhtml(valores[5]) # quitar las etiquetas de html del texto que describe la incidencia 
        if valores[2] != '':  # En caso de que exista una hora fin
            mensaje = "Hubo un atasco en la carretera " + valores[4]
            fin = " el cual terminó a las " + valores[2] + " el día " + valores[3]
        else: 
            mensaje = "Existe un atasco en la carretera " + valores[4]
        mensaje = mensaje + " que inicio a las " + valores[0] + " el día " + valores[1]
        mensaje = mensaje + fin
        mensaje = mensaje + " con la siguientes características:\n" +  cleantext

        print(mensaje+"\n")
    
